In [1]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
import dill
import numpy as np
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2

In [2]:
import qiskit_nature
qiskit_nature.__version__

'0.7.2'

In [3]:
BL = 0.6

In [4]:
# Parameters
BL = 0.35


### 1 Construct the Hamiltonian

Helper function to construct the Hamiltonian represented as a Pauli decomposition stored in `qubit_op`, while also returning the `nuclear_repulsion_energy` and `core_electron_energy`.

In [5]:
def get_qubit_op(BL):
    molecule = MoleculeInfo(
        symbols=["H", "H"],
        coords=([0.0, 0.0, 0.0], [BL, 0.0, 0.0]),
        multiplicity=1,
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    properties = driver.run()

    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)

    # Get the fermionic Hamiltonian (first element)
    fermionic_op, _ = problem.second_q_ops()

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    # Use Jordan-Wigner (no tapering)
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(fermionic_op)
    # converter = QubitConverter(mapper=mapper)
    # qubit_op = converter.convert(fermionic_op, num_particles=num_particles)

    core_electron_energy = problem.hamiltonian.constants["FreezeCoreTransformer"]
    nuclear_repulsion_energy = problem.nuclear_repulsion_energy

    return (
        qubit_op,
        num_particles,
        num_spatial_orbitals,
        problem,
        mapper,
        nuclear_repulsion_energy,
        core_electron_energy,
    )


In [6]:
(qubit_op, num_particles, num_spatial_orbitals, problem, mapper,nuclear_repulsion_energy, core_electron_energy) = get_qubit_op(BL)

In [7]:
qubit_op

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.52499962+0.j,  0.24679198+0.j, -0.50062384+0.j,  0.24679198+0.j,
 -0.50062384+0.j,  0.14283108+0.j,  0.18619134+0.j,  0.18347439+0.j,
  0.04064331+0.j,  0.04064331+0.j,  0.04064331+0.j,  0.04064331+0.j,
  0.18347439+0.j,  0.19386107+0.j,  0.14283108+0.j])

In [8]:
# Extract terms and coefficients
pauli_list = list(zip(qubit_op.paulis, qubit_op.coeffs))

# Sort terms by absolute coefficient magnitude (largest first)
sorted_pauli_list = sorted(pauli_list, key=lambda x: abs(x[1]), reverse=True)

# Print sorted terms
# for pauli, coef in sorted_pauli_list:
#     print(f"Pauli: {pauli}, Coefficient: {coef}")

# Save the list
with open("sorted_pauli_list.pkl", "wb") as f:
    dill.dump(sorted_pauli_list, f)

In [9]:
dd = np.real(min(np.linalg.eigvals(qubit_op.to_matrix())))

### 2 Perform VQE

Define a class to perform VQE.

In [10]:
# Define the custom VQE
class CustomVQE:
    def __init__(self, estimator, ansatz, optimizer, qubit_op, dd, NREplusCEE, custom_function, initial_point=None):
        self.estimator = estimator
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.qubit_op = qubit_op
        self.NREplusCEE = NREplusCEE
        self.dd = dd
        self.custom_function = custom_function
        self.initial_point = initial_point  or [0] * ansatz.num_parameters

    def compute_minimum_eigenvalue(self):
        # Define the function to minimize
        def objective_function(params):
            # Evaluate the expectation value of the qubit operator
            expectation = self.estimator.run([self.ansatz], [self.qubit_op], [params]).result().values[0]
            # Evaluate the custom objective function
            return self.custom_function(expectation, params)

        # Use the `minimize` method for optimization
        opt_result = self.optimizer.minimize(
            fun=objective_function,
            x0=self.initial_point
        )

        result = {
            'x': [float(value) for value in opt_result.x],
            'fun': opt_result.fun,
            'dd': self.dd,
            'NREplusCEE': self.NREplusCEE
        }

        
        return result


In [11]:
# l=12 and maxiter = 200 works for LiH and HF

In [12]:
optimizer = SLSQP(maxiter=300)
noiseless_estimator = Estimator(approximation=True)


# Define ansatz
var_form = EfficientSU2(qubit_op.num_qubits, reps=12, entanglement='full', skip_unentangled_qubits=False, parameter_prefix='a')


# Example Custom Objective Function
def custom_objective_function(expectation_value, parameters):
    # penalty_term = 0.1 * sum(param**2 for param in parameters)  # Example regularization
    return expectation_value 


# Create an instance of CustomVQE
custom_vqe = CustomVQE(
    estimator=noiseless_estimator,
    ansatz=var_form,
    optimizer=optimizer,
    qubit_op=qubit_op,
    dd = dd,
    NREplusCEE = nuclear_repulsion_energy + core_electron_energy,
    custom_function=custom_objective_function,
    initial_point = list(np.random.uniform(-np.pi, np.pi, var_form.num_parameters))
)


In [13]:
vqe_result = custom_vqe.compute_minimum_eigenvalue()

In [14]:
vqe_result

{'x': [-0.24387998531237715,
  -1.8500661632987114,
  -3.059338137646308,
  -0.12586621560854439,
  -1.7435897088746126,
  -1.8776163165667987,
  2.7711269536688827,
  -0.041834803909828855,
  -1.800250490525609,
  0.4525263224832373,
  2.609061377151411,
  -1.8326184742470422,
  0.37521569760297246,
  0.06532373732925252,
  1.1268397243132757,
  -0.639640706341535,
  1.7781224198680974,
  -1.8780695778734657,
  0.4711448398085903,
  1.593113875870166,
  2.072880776787573,
  1.198492275810608,
  1.1688993556939606,
  -1.0223620452793285,
  -0.5818029893400073,
  2.1655552236894664,
  1.899617914839223,
  -2.2412746141167315,
  -0.5662767376501092,
  2.5971794903871337,
  -0.03961043156814824,
  1.6644799697064612,
  1.4795061721907512,
  -2.6218842025642703,
  0.30171966386173105,
  -2.5656089781774867,
  -0.15184983257167434,
  2.188653022224701,
  2.0629790779794765,
  -3.2373513787615065,
  -1.133689827701317,
  1.0064288080321828,
  2.717490244396952,
  -2.759305567245771,
  2.6585

In [15]:
len(vqe_result['x'])

104

In [16]:
print(vqe_result['fun'] - vqe_result['dd'])

1.0795989968670483e-07


In [17]:
with open("vqe_result.pkl", "wb") as f:
    dill.dump(vqe_result, f)